In [ ]:
from gprm.datasets import Paleogeography
from gprm import PointDistributionOnSphere
import pygplates
import pygmt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import matplotlib as mpl
mpl.rc('font',family='sans-serif')
mpl.rcParams['axes.linewidth'] = 2
mpl.rcParams['xtick.major.width'] = 2
mpl.rcParams['ytick.major.width'] = 2



Paleomap = Paleogeography.fetch_Paleomap(resolution='01d')

equal_area_points = PointDistributionOnSphere(distribution_type='healpix',N=64)
N_equalarea_pts = equal_area_points.latitude.shape[0]
area_of_one_point = (4 * np.pi * pygplates.Earth.mean_radius_in_kms**2)/N_equalarea_pts/1e6


In [ ]:
cat_sym = []
cat_asym = []
hyp_cat = []
mountain_cat = []

for reconstruction_time in Paleomap.keys():

    grd_ea = pygmt.grdtrack(grid=Paleomap[reconstruction_time], 
                         points=pd.DataFrame(data={'x': equal_area_points.longitude,
                                                   'y': equal_area_points.latitude}),
                         newcolname='z',
                         no_skip=True)['z']
    
    hyp_cat.append(np.histogram(grd_ea, bins=np.arange(0,10001,50))[0])
    mountain_cat.append(np.histogram(grd_ea, bins=np.arange(100,6001,500))[0])
  
    ind = np.where(grd_ea>1000.)
 
    result = np.histogram(equal_area_points.latitude[ind],np.arange(-90.,91.,10.))    
    cat_asym.append(result[0]*area_of_one_point)
    
    result = np.histogram(np.abs(equal_area_points.latitude[ind]),np.arange(0.,91.,10.))    
    cat_sym.append(result[0]*area_of_one_point)

    #break

In [ ]:
cm = plt.cm.get_cmap('viridis_r')
color_list = cm(np.linspace(0,1,9))
labels=['{:d}-{:d} latitude'.format(int(lat),int(lat+10)) for lat in np.linspace(0,90,10)]

fig = plt.figure(figsize=(9,4.5))
plt.stackplot(list(Paleomap.keys()),
              np.vstack(cat_sym).T*100./510., # * 1e6 * 4 * np.pi / pygplates.Earth.mean_radius_in_kms**2, 
              colors=color_list, labels=labels)
plt.xlim(541,0)
#plt.ylim(0,140)
plt.grid()
plt.xlabel('Reconstruction Time (Ma)')
plt.ylabel('Area $(10^6 km^2)$')
plt.legend(ncol=3)
plt.savefig('../images/Mountain_Latitude_Ranges_Phanerozoic_Stackplot.png', dpi=600)
plt.show()


In [ ]:
cm = plt.cm.get_cmap('viridis_r')
color_list = cm(np.linspace(0,1,9))
labels=['{:d}-{:d} latitude'.format(int(lat),int(lat+10)) for lat in np.linspace(0,90,10)]

fig = plt.figure(figsize=(9,6))

ax1 = fig.add_axes([0.1,0.5,0.8,0.4])
ax1.stackplot(list(Paleomap.keys()),
              np.vstack(cat_asym).T[9:,:], colors=color_list, labels=labels)
ax1.set_ylim(0,2200*area_of_one_point)
ax1.set_xlim(541,0)
ax1.grid()
ax1.xaxis.set_ticklabels([])
ax1.set_ylabel('Area North of Equator $(10^6 km^2)$')

ax2 = fig.add_axes([0.1,0.1,0.8,0.4])
ax2.stackplot(list(Paleomap.keys()),
              np.flipud(np.vstack(cat_asym).T[:9,:]), colors=color_list)
ax2.set_ylim(2200*area_of_one_point,0)
ax2.grid()
ax2.set_xlim(541,0)
ax2.set_ylabel('Area South of Equator $(10^6 km^2)$')
ax2.set_xlabel('Reconstruction Time (Ma)')

ax1.legend(ncol=3, loc='upper left')

plt.savefig('../images/Mountain_Latitude_Ranges_Phanerozoic_Stackplot_Asymmetric.png', dpi=600)
plt.show()

In [ ]:

data = np.flipud(np.cumsum(np.flipud(np.vstack(hyp_cat).T),axis=0)).astype('float')
data[data==0] = np.nan

fig,ax = plt.subplots(figsize=(9,4.5))
cm = plt.contourf(Paleomap.keys(),
             np.arange(50,10001,50),
             data, 
             levels=np.arange(0,10001,1000),
            cmap='bone_r', extend='max')
ax.set_ylim(0,4000)
ax.set_facecolor('lightgrey')

ax.set_ylabel('Elevation [m]')
ax.set_xlabel('Reconstruction Time (Ma)')

plt.colorbar(cm, label='Land area above elevation contour $[10^6 km^2]$')

plt.savefig('../images/Mountain_area_above_contour_Phanerozoic.png')
plt.show()

In [ ]:
data = np.flipud(np.cumsum(np.flipud(np.vstack(hyp_cat).T),axis=0)).astype('float')
data[data==0] = np.nan
fig,ax = plt.subplots()
cm = ax.pcolormesh(Paleomap.keys(),
             np.arange(0,10001,50),
             data,
            cmap=plt.cm.get_cmap('PuBuGn',10), vmin=250, vmax=10000)
ax.set_ylim(0,8000)
ax.set_facecolor('lightgrey')
plt.colorbar(cm)


In [ ]:
np.sum(np.isnan(data))

In [ ]:
cm = plt.cm.get_cmap('cubehelix_r')
color_list = cm(np.linspace(0.1,1,12))

data = np.vstack(mountain_cat).T
plt.stackplot(list(Paleomap.keys()),
              data*100./510., # * 1e6 * 4 * np.pi / pygplates.Earth.mean_radius_in_kms**2, 
              colors=color_list, labels=labels, baseline='zero')
plt.show()


In [ ]:
cm = plt.cm.get_cmap('cubehelix_r')
color_list = cm(np.linspace(0.1,1,12))
labels=['{:d}-{:d} m'.format(int(elev),int(elev+500)) for elev in np.arange(0,6001,500)]

fig = plt.figure(figsize=(9,10))

ax1 = fig.add_axes([0.1,0.5,0.8,0.4])
ax1.stackplot(list(Paleomap.keys()),
              np.vstack(mountain_cat).T[1:,:], colors=color_list[1:], labels=labels)
ax1.set_ylim(0,1100000*area_of_one_point)
ax1.set_xlim(541,0)
ax1.grid()
ax1.xaxis.set_ticklabels([])
ax1.set_ylabel('Area of highlands by contour range $(10^6 km^2)$')

ax2 = fig.add_axes([0.1,0.1,0.8,0.4])
ax2.stackplot(list(Paleomap.keys()),
              np.vstack(mountain_cat).T[:1,:], colors=color_list)
ax2.set_ylim(1100000*area_of_one_point,0)
ax2.grid()
ax2.set_xlim(541,0)
ax2.set_ylabel('Area of lowlands $(10^6 km^2)$')
ax2.set_xlabel('Reconstruction Time (Ma)')

ax1.legend(ncol=3, loc='upper left')

plt.savefig('../images/Mountain_Contour_Ranges_Phanerozoic_Stackplot_Asymmetric.png', dpi=600)
plt.show()
